In [1]:
import numpy as np
import scipy.sparse as sparse
import scipy.linalg as linalg
from scipy.sparse.linalg import expm
import matplotlib.pyplot as plt
import codetiming


In [2]:
N = 1000
np.random.seed(0)
A = np.random.uniform(size=(N,N))
A[A<0.98] = 0
As = sparse.lil_matrix(A,shape=(N,N))

In [3]:
t1 = codetiming.Timer()
t1.start()
L, U = np.linalg.eig(A)
Uinv = np.linalg.inv(U)
t1.stop()

Elapsed time: 2.7983 seconds


2.7982565000000363

In [10]:
# Load data
data = np.load('./M_example_lmax8.npz')
M = data['M']

# Test time
times = np.logspace(-8,-2,128)

# Fake initial conditions
c0 = np.random.uniform(size=(M.shape[0]))
c0[c0<0.95] = 0

# Sparse matrix exponentiation
t3 = codetiming.Timer()
t3.start()
Ms = sparse.csc_matrix(M)
for t in times:
    Es = expm(Ms*t)
    ci = Es.dot(c0)
t3.stop()

# Normal diagonalization plus exponential
# https://www.benjaminjohnston.com.au/matmul
t1 = codetiming.Timer()
t1.start()
L, U = np.linalg.eig(M)
Uinv = np.linalg.inv(U)
for t in times:
    ci = np.matmul(U, np.diag(np.exp(L*t)))
    ci = np.matmul(ci, Uinv)
    ci = ci.dot(c0)
t1.stop()


# Sparse matrix exponentiation
t3 = codetiming.Timer()
t3.start()
for t in times:
    Es = linalg.expm(M*t)
    ci = Es.dot(c0)
t3.stop()

# Normal diagonalization plus exponential transposed
# https://www.benjaminjohnston.com.au/matmul
t1 = codetiming.Timer()
t1.start()
L, U = np.linalg.eig(M)
Uinv = np.linalg.inv(U)
ci = c0.dot(Uinv.T)
for t in times:
    ci = np.matmul(ci, np.diag(np.exp(L*t)))
    ci = np.matmul(ci, U.T)
t1.stop()

# The sparse.linalg.expm algoritm is faster for large lmax for several up to evaluations
# The linalg.expm algoritm is faster for 1 or 2 evaluations

Elapsed time: 5.6517 seconds
Elapsed time: 0.2789 seconds
Elapsed time: 3.0950 seconds
Elapsed time: 0.0831 seconds


0.08306629999992765

In [62]:
Es

array([[3.42670994e-30, 0.00000000e+00, 0.00000000e+00, ...,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       [0.00000000e+00, 4.27093179e-91, 0.00000000e+00, ...,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       [0.00000000e+00, 0.00000000e+00, 1.56474721e-38, ...,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       ...,
       [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00]])

In [63]:
# Try the sparse matrix multiplication
t1 = codetiming.Timer()
t1.start()
L, U = np.linalg.eig(M)
Uinv = np.linalg.inv(U)
Us = sparse.bsr_matrix(U)
Uinvs = sparse.bsr_matrix(Uinv)
for t in times:
    ci = Us.dot(sparse.diags(np.exp(L*t)))
    ci = ci.dot(Uinvs)
t1.stop()

Elapsed time: 2.6191 seconds


2.6191108000002714

In [64]:
# Test of multiplying by c0 before hand can speed up the calculation by increasing sparsity

# Load data
data = np.load('../M_example_lmax20.npz')
M = data['M']

# Test time
times = np.logspace(-8,-2,1)


c0 = np.zeros([M.shape[0]])
c0[0] = 1

# Normal diagonalization plus exponential
# https://www.benjaminjohnston.com.au/matmul
t1 = codetiming.Timer()
t1.start()
L, U = np.linalg.eig(M)
Uinv = np.linalg.inv(U)
for t in times:
    cin = np.matmul(U, np.diag(np.exp(L*t)))
    cin = np.matmul(cin, Uinv)
    cin = cin.dot(c0)
t1.stop()

# Normal diagonalization plus exponential transposed
# https://www.benjaminjohnston.com.au/matmul
t1 = codetiming.Timer()
t1.start()
L, U = np.linalg.eig(M)
Uinv = np.linalg.inv(U)
ci = c0.dot(Uinv.T)
for t in times:
    ci = np.matmul(ci, np.diag(np.exp(L*t)))
    ci = np.matmul(ci, U.T)
t1.stop()
print(np.allclose(cin, ci))

# # Normal diagonalization plus exponential transposed and sparse
# # https://www.benjaminjohnston.com.au/matmul
# t1 = codetiming.Timer()
# t1.start()
# L, U = np.linalg.eig(M)
# Uinv = np.linalg.inv(U)
# ci = c0.dot(Uinv.T)
# Us = sparse.csc_matrix(U)
# for t in times:
#     ci = ci.dot(sparse.diags(np.exp(L*t)))
#     print(ci.shape)
#     ci = ci.dot(Us.T)
# t1.stop()
# print(np.allclose(cin, ci))

Elapsed time: 2.4273 seconds
Elapsed time: 2.2354 seconds
True


In [65]:
ci.shape

(1323,)